# Prompt Engineering with Amazon Bedrock

In this notebook, you explore different prompt engineering techniques. A *prompt* is the message sent to a model to generate a response. Prompt engineering might involve:
* Word choice
* Phrasing
* Providing additional information
* Providing examples

1. **Generate recommendations based on metadata.**
    - **Task:** Text generation
    - **Prompt technique:** Zero-shot
2. **Estimate audience for TV shows based on historical data.**
    - **Task:** Complex reasoning
    - **Prompt technique:** Chain-of-thought (CoT)
3. **Create a question-answering assistant.**
    - **Task:** Question answering with a dialogue assistant (without memory)
    - **Prompt technique:** Few-shot
4. **Create splash pages that describe upcoming events.**
    - **Task:** Code generation
    - **Prompt technique:** Zero-shot

### Import libraries and create an Amazon Bedrock client.

First, import the needed libraries and create an Amazon Bedrock Boto client. Then, list the available models from Amazon Bedrock.

In [2]:
## Code Cell 1 ##

import boto3
import json
import csv
from datetime import datetime


bedrock = boto3.client('bedrock')
bedrock_runtime = boto3.client('bedrock-runtime')
bedrock.list_foundation_models()

{'ResponseMetadata': {'RequestId': 'ddd8e497-1539-4675-8739-ce750cc503c8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sun, 27 Oct 2024 20:36:10 GMT',
   'content-type': 'application/json',
   'content-length': '31106',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ddd8e497-1539-4675-8739-ce750cc503c8'},
  'RetryAttempts': 0},
 'modelSummaries': [{'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-tg1-large',
   'modelId': 'amazon.titan-tg1-large',
   'modelName': 'Titan Text Large',
   'providerName': 'Amazon',
   'inputModalities': ['TEXT'],
   'outputModalities': ['TEXT'],
   'responseStreamingSupported': True,
   'customizationsSupported': [],
   'inferenceTypesSupported': ['ON_DEMAND'],
   'modelLifecycle': {'status': 'ACTIVE'}},
  {'modelArn': 'arn:aws:bedrock:us-east-1::foundation-model/amazon.titan-image-generator-v1:0',
   'modelId': 'amazon.titan-image-generator-v1:0',
   'modelName': 'Titan Image Generator G1',
   'providerName': 'Amazon',

### Create a help function for calling Amazon Bedrock. 
Define a utility function for calling Amazon Bedrock to help pass the proper body, depending on the invoked model.

In [3]:
## Code Cell 2 ##

def call_bedrock(modelId, prompt_data): 
    if 'amazon' in modelId:
        body = json.dumps({
            "inputText": prompt_data,
            "textGenerationConfig":
            {
                "maxTokenCount": 1024,
                "stopSequences":[],
                "temperature":0.7,
                "topP":0.9
            }
        })
    elif 'meta' in modelId:
        body = json.dumps({
            "prompt": prompt_data,
            "max_tokens_to_sample": 4096,
            "stop_sequences":[],
            "temperature":0,
            "top_p":0.9
        })
    elif 'mistral' in modelId:
        body = json.dumps({
            "prompt": prompt_data,
            "max_tokens_to_sample": 4096,
            "stop_sequences":[],
            "temperature":0,
            "top_p":0.9
        })
        print('Parameter model must be one of Titan, Lama, or Mixtral')
        return
    accept = 'application/json'
    contentType = 'application/json'

    before = datetime.now()
    response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    latency = (datetime.now() - before)
    response_body = json.loads(response.get('body').read())

    if 'amazon' in modelId:
        response = response_body.get('results')[0].get('outputText')
    elif 'meta' in modelId:
        response = response_body.get('completion')
    elif 'mistral' in modelId:
        response = response_body.get('completions')[0].get('data').get('text')
        

    return response, latency

Now, you're ready to run examples with different prompt engineering techniques. 

-----

## 1. Generate recommendations based on metadata.

**Use case:** A media and entertainment company wants to generate TV show recommendations for their customers based on viewer metadata, sucha as country, age range, and theme.

**Task:** Text generation

**Prompt technique:** Zero-shot

**Model:** Amazon Titan

A single prompt is comprised of a number of different components, wnich might include:
* A task or instruction
* Context of a task, such as a description of the relevant domain
* Demonstration examples
* Input text, which you want Amazon Titan to use in forming a response

Your prompt will be a combination of one or more of these components, depending on the use case, the availability of the data, and the task.

In [4]:
## Code Cell 3 ##

prompt_data ="""
Human:
Generate a list of 10 recommended TV shows to watch, considering the information in the <metadata></metadata> XML tags. Include a very brief description of each recommendation.

<metadata>
Country is UK
Age range between 20-30
Shows must be about sports
</metadata>

Assistant:
"""


response, latency = call_bedrock('amazon.titan-text-premier-v1:0', prompt_data)
print(response, "\n\n", "Inference time:", latency)

1. "Sunderland 'Til I Die" (2018-Present) - An enthralling docuseries following the trials and tribulations of Sunderland A.F.C. as they navigate the lower divisions of English football.

2. "The Premier League Show" (2016-Present) - A comprehensive weekly program offering in-depth analysis, interviews, and features on the latest happenings in the English Premier League.

3. "A League of Their Own" (2010-Present) - A lighthearted panel show centered around sports, hosted by James Corden and featuring celebrity guests and sporting legends.

4. "Match of the Day" (1964-Present) - A long-running, iconic football highlights show that recaps the latest matches from the Premier League and other top-tier competitions.

5. "Super League Show" (1999-Present) - A weekly show dedicated to the Rugby Super League, featuring match highlights, expert analysis, and exclusive interviews.

6. "Football Focus" (1974-Present) - A weekly football preview show that provides expert analysis, interviews, and 

------

## 2. Estimate audience for TV shows based on historical data.

**Use case:** The company wants to estimate the next day's audience levels based on historical information and a TV show's metadata.

**Task:** Complex reasoning

**Prompt technique:** Chain-of-thought (CoT)

**Model:** Amazon Titan

In [5]:
## Code Cell 4 ##

prompt_data ="""
Human: Last week, three television channels had the following viewership data:
- Monday: SportsTV 6500, NewsTV 3200, EntertainmentTV 4150
- Tuesday: SportsTV 6400, NewsTV 3300, EntertainmentTV 4100
- Wednesday: SportsTV 6300, NewsTV 3400, EntertainmentTV 4250

Question: How many viewers can we expect next Friday on SportsTV?
Answer: According to the numbers given, and without having more information, there is a daily decrease of 100 viewers for SportsTV.
If we assume that this trend will continue for the next few days, we can expect 6200 viewers for the next day, which is Thursday, and
therefore 6100 viewers for Friday.

Question: How many viewers can we expect on Saturday for each of the three channels? Think step-by-step, and provide recommendations for increasing the viewers.
Assistant:
Answer:
"""

response, latency = call_bedrock('amazon.titan-text-premier-v1:0', prompt_data)
print(response, "\n\n", "Inference time:", latency)

On Saturday, assuming the same trend, we can expect the following viewership:
- SportsTV: 6000
- NewsTV: 3500
- EntertainmentTV: 4350

To increase viewership, I would recommend the following:

1. SportsTV: Organize a special event or live game to attract more viewers.
2. NewsTV: Improve the content and format of the news program to make it more engaging.
3. EntertainmentTV: Offer exclusive content or collaborations with popular artists to attract more viewers.

Note: These are just recommendations based on the data provided, and the actual viewership may vary depending on various factors. 

 Inference time: 0:00:03.019517


------

## 3. Create a question-answering assistant.

**Use case:** The company wants to create a dialogue assistant that is capable of answering questions about available TV shows based on show data.

**Task:** Question answering with a dialogue Assistant (no memory)

**Prompt technique:** Few-shot

**Model:** Amazon Titan

In [6]:
## Code Cell 5 ##

prompt_data ="""
Context: The shows available are as follows
1. Circus, showing at the Plaza venue, assigned seating, live at 8pm on weekends
2. Concert, showing at the Main Theater, assigned seating, live at 10pm everyday
3. Basketball tricks, showing at the Sports venue, standing seating, live at 5pm on weekdays

Instruction: Answer any questions about the available shows. If you don't know the answer, say 'Apologies, I don't have the answer for that. Please contact our team by phone.'

Assistant: Welcome to Entertainment Tonight, how can I help you?
Human: Hi, I would like to know what shows are available please.
Assistant: Of course. Right now, we have the Circus, the Concert, and the Basketball tricks shows.
Human: Thank you. I would like to know when and where those shows are available please.
Assistant:
"""

response, latency = call_bedrock('amazon.titan-text-premier-v1:0', prompt_data)
print(response, "\n\n", "Inference time:", latency)

The Circus is showing at the Plaza venue, live at 8pm on weekends. The Concert is showing at the Main Theater, live at 10pm everyday. The Basketball tricks is showing at the Sports venue, live at 5pm on weekdays. 

 Inference time: 0:00:01.208994


------

## 4. Create splash pages that describe upcoming events.

**Use case:** The company wants a quick, efficient way to create HTML pages to promote upcoming events.

**Task:** Code generation

**Prompt technique:** Zero-shot

**Model:** Amazon Titan

In [8]:
## Code Cell 6 ##

prompt_data ="""
An upcoming music concert is presented by the company, Music Promotions.
The event targets a young audience, age range between 18 and 40.
The event will occur in the Royal Music Theater.
Seating is assigned and tickets can be purchased through the Music Promotions website.
The event is a music concert performed by the band, Super Rockers.
The event will occur on June 30, 2023, and doors will open at 20:00.

Based on the information provided above, generate the HTML code for an attractive splash page to promote the event.
"""

response, latency = call_bedrock('amazon.titan-text-premier-v1:0', prompt_data)
print(response, "\n\n", "Inference time:", latency)
from IPython.display import display, HTML
display(HTML(response))


<!DOCTYPE html>
<html>
<head>
<title>Super Rockers Live in Concert</title>
<meta charset="UTF-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<style>
body {
font-family: Arial, sans-serif;
background-color: #f0f0f0;
color: #333;
}
.container {
max-width: 800px;
margin: 0 auto;
padding: 20px;
background-color: #fff;
box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
}
h1 {
text-align: center;
color: #007BFF;
margin-bottom: 30px;
}
p {
margin-bottom: 20px;
}
.info-box {
background-color: #F5F5F5;
padding: 20px;
margin-bottom: 20px;
border-radius: 5px;
box-shadow: 0 0 5px rgba(0, 0, 0, 0.1);
}
.info-box h2 {
margin-top: 0;
margin-bottom: 10px;
}
.button {
display: inline-block;
background-color: #007BFF;
color: #fff;
padding: 10px 20px;
border-radius: 5px;
text-decoration: none;
margin-right: 10px;
}
.button:hover {
background-color: #0056b3;
}
</style>
</head>
<body>
<div class="container">
<h1>Super Rockers Live in Concert</h1>
<div class="info-box">
<h2>About the Event</

------
## DIY – Use a prompt template to improve the application prompt. 
------

1. Use the Code Cell 7 call_bedrock Python function to update the invoke_bedrock Lambda function. 
2. After updating the call_bedrock function, redeploy the function by using the same steps that you used during the Practice section. 
3. Test the application again, asking for a TV show recommendation, to see how the model response improved with the user metadata.


In [9]:
# Code Cell 7

def call_bedrock(modelId, prompt_data):
    bedrock_runtime = boto3.client('bedrock-runtime')
    
    prompt_template = """
    User Metadata:
    Name: John Doe
    Age: 35
    Location: New York, USA
    Interests: Technology, Artificial Intelligence, Music
    Occupation: Software Engineer
    
    Question: {prompt_data}
    """.format(prompt_data=prompt_data)
    
    body = json.dumps({
        "inputText": prompt_template,
        "textGenerationConfig": {
            "maxTokenCount": 1000,
            "stopSequences": [],
            "temperature": 0.7,
            "topP": 0.9
        }
    })
    print("bedrock-input:",body)

    accept = 'application/json'
    contentType = 'application/json'

    before = datetime.now()
    response = bedrock_runtime.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
    latency = (datetime.now() - before).seconds
    response_body = json.loads(response.get('body').read())
    response = response_body.get('results')[0].get('outputText')
    
    return {
        'latency': str(latency),
        'response': response
    }